In [1]:
# !pip install torch torchvision torchaudio transformers datasets scikit-learn


In [1]:
# import torch
# import accelerate
# import transformers

# print("Torch version:", torch.__version__)
# print("Torch CUDA available:", torch.cuda.is_available())
# print("Accelerate version:", accelerate.__version__)
# print("Transformers version:", transformers.__version__)


In [2]:
from datasets import load_dataset

ds = load_dataset("stanfordnlp/imdb")

In [3]:
ds['train']['text'][0]

'I rented I AM CURIOUS-YELLOW from my video store because of all the controversy that surrounded it when it was first released in 1967. I also heard that at first it was seized by U.S. customs if it ever tried to enter this country, therefore being a fan of films considered "controversial" I really had to see this for myself.<br /><br />The plot is centered around a young Swedish drama student named Lena who wants to learn everything she can about life. In particular she wants to focus her attentions to making some sort of documentary on what the average Swede thought about certain political issues such as the Vietnam War and race issues in the United States. In between asking politicians and ordinary denizens of Stockholm about their opinions on politics, she has sex with her drama teacher, classmates, and married men.<br /><br />What kills me about I AM CURIOUS-YELLOW is that 40 years ago, this was considered pornographic. Really, the sex and nudity scenes are few and far between, ev

In [4]:
# from PIL import Image
# print(Image.__version__)


In [5]:
from transformers import AutoTokenizer
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

tokenizer = AutoTokenizer.from_pretrained("google-bert/bert-base-cased")



def tokenize(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

In [6]:
import torch
print("CUDA available:", torch.cuda.is_available())
print("GPU count:", torch.cuda.device_count())
if torch.cuda.is_available():
    print("GPU name:", torch.cuda.get_device_name(0))


CUDA available: True
GPU count: 1
GPU name: NVIDIA GeForce RTX 4060 Laptop GPU


In [7]:
dataset = ds.map(tokenize, batched=True)

In [8]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 50000
    })
})

In [9]:
ds_train = dataset['train'].shuffle(seed=42)
ds_test = dataset['test'].shuffle(seed=42)

In [10]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("google-bert/bert-base-cased", num_labels=2)

model.to(device)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [11]:
# !pip install evaluate

In [12]:
import numpy as np
import evaluate

metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    # convert the logits to their predicted class
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [13]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="imdb_classifier",
    eval_strategy="epoch",
    push_to_hub=False,
)

In [ ]:
!wandb login

In [14]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    compute_metrics=compute_metrics
)
trainer.train()

wandb: Currently logged in as: sarbojitbiswas744 (sarbojitbiswas744-alliance-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss,Accuracy
1,0.311000,0.383269,0.884640
2,0.213000,0.318977,0.917840
3,0.104000,0.374872,0.926800


TrainOutput(global_step=9375, training_loss=0.23197329752604168, metrics={'train_runtime': 7748.3128, 'train_samples_per_second': 9.68, 'train_steps_per_second': 1.21, 'total_flos': 1.9733329152e+16, 'train_loss': 0.23197329752604168, 'epoch': 3.0})

In [15]:
metrics = trainer.evaluate()
print(metrics)

socket.send() raised exception.
socket.send() raised exception.
socket.send() raised exception.
socket.send() raised exception.
socket.send() raised exception.
socket.send() raised exception.
socket.send() raised exception.
socket.send() raised exception.
socket.send() raised exception.
socket.send() raised exception.
socket.send() raised exception.
socket.send() raised exception.
socket.send() raised exception.
socket.send() raised exception.
socket.send() raised exception.
socket.send() raised exception.
socket.send() raised exception.
socket.send() raised exception.
socket.send() raised exception.
socket.send() raised exception.
socket.send() raised exception.
socket.send() raised exception.
socket.send() raised exception.
socket.send() raised exception.
socket.send() raised exception.
socket.send() raised exception.
socket.send() raised exception.
socket.send() raised exception.
socket.send() raised exception.
socket.send() raised exception.


{'eval_loss': 0.37487199902534485, 'eval_accuracy': 0.9268, 'eval_runtime': 3608.5513, 'eval_samples_per_second': 6.928, 'eval_steps_per_second': 0.866, 'epoch': 3.0}


In [16]:
trainer.save_model("./bert-imdb-sentiment-model")

In [17]:
text = "This movie was awesome! The acting was great, plot was wonderful, and there were pythons...so yea!"
inputs = tokenizer(text, padding=True, truncation=True, return_tensors="pt")
preds = model(**inputs.to(device)).logits.argmax(-1).item()
print(preds)  # 1 is positive, 0 is negative
print("Positive" if preds else "Negative")


1
Positive
